# Pipeline de Validación de Datos con TensorFlow Data Validation (TFDV)

## 1. Contexto del Proyecto

Este proyecto tiene como objetivo crear un ambiente de desarrollo para machine learning que permita la ingesta, validación y transformación de datos. En este ejemplo se trabaja con un dataset de **cubierta forestal**, en el cual se predice el tipo de cubierta forestal ("Cover_Type") a partir de variables cartográficas. La canalización incluye:

- **Selección de características:** Se elige un subconjunto de variables numéricas junto con la etiqueta.
- **Ingesta y división del dataset:** Se carga el archivo CSV, se visualizan los datos y se divide en conjuntos de entrenamiento y prueba.
- **Generación de estadísticas:** Se analizan las distribuciones de las variables mediante TFDV.
- **Inferencia y curado del esquema:** Se infiere un esquema a partir de las estadísticas y se ajustan dominios esperados para ciertas características.
- **Detección de anomalías:** Se valida un conjunto de datos (simulando un escenario de inferencia en el que falta la etiqueta) para detectar discrepancias.

Este pipeline forma parte de un ambiente reproducible que facilita la integración de metadatos y el seguimiento de la procedencia de los datos, asegurando la calidad antes de proceder a etapas posteriores como la transformación o el entrenamiento del modelo.


## 2. Importación de librerías

In [1]:
# Import libraries
import os
import requests
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
from sklearn.model_selection import train_test_split

2025-02-26 17:03:48.206088: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-26 17:03:48.235486: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-26 17:03:48.451791: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-26 17:03:48.705403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-26 17:03:48.896518: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

# 3. Carga del Dataset

Se descarga el dataset de cubierta forestal (si no existe en el directorio) y se carga en un DataFrame de Pandas.

In [28]:
## Download data
if not os.path.isfile('cubierta_forestal.csv'):
    #http://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008#
    url = 'https://docs.google.com/uc?export= \
download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True)
    open('cubierta_forestal.csv', 'wb').write(r.content)

In [19]:
# Check some data
df = pd.read_csv('cubierta_forestal.csv', index_col=False)
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


In [20]:
# Check all columns names
df.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type',
       'Cover_Type'],
      dtype='object')

## 4. Selección de Características

Se selecciona un subconjunto del dataset que contenga únicamente las características numéricas relevantes y la etiqueta "Cover_Type".

In [21]:
# Se elige un subconjunto que contenga solo características numéricas y la etiqueta (Cover_Type)
selected_columns = [
    'Elevation',
    'Slope',
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am',
    'Hillshade_Noon',
    'Horizontal_Distance_To_Fire_Points',
    'Cover_Type'
]

df = df[selected_columns]

## 5. División del Conjunto de Datos

Se divide el DataFrame en conjuntos de entrenamiento y prueba para analizar la distribución de los datos y simular escenarios de inferencia.

In [ ]:
# --- Dividir el DataFrame en conjuntos de entrenamiento y prueba ---
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

## 6. Generación de Estadísticas del Conjunto de Entrenamiento

Utilizando TFDV se generan y visualizan las estadísticas del conjunto de entrenamiento. Esto permite conocer la distribución de cada variable.

In [22]:
# --- 2.1. Generar estadísticas del conjunto de entrenamiento ---
stats_train = tfdv.generate_statistics_from_dataframe(train_df)
# Visualización interactiva (útil en entornos Jupyter)
tfdv.visualize_statistics(stats_train)


# 7. Inferencia del Esquema

A partir de las estadísticas generadas se infiere un esquema del dataset que servirá como referencia para validar los datos.

In [23]:
# --- 2.2. Inferir un esquema a partir de las estadísticas de entrenamiento ---
schema = tfdv.infer_schema(stats_train)
tfdv.display_schema(schema)


,Type,Presence,Valency,Domain
Feature name,,,,
'Elevation',INT,required,,-
'Slope',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Vertical_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Cover_Type',INT,required,,-


## 8. Curado del Esquema con Dominios Específicos

Se actualiza el esquema para especificar los rangos esperados de las variables clave:

- Hillshade_9am y Hillshade_Noon: Valores entre 0 y 255.

- Slope: Valores entre 0 y 90.

- Cover_Type: Valores entre 0 y 6 (variable categórica).

Se utiliza tfdv.get_feature() para acceder a cada característica y ajustar sus dominios.

In [24]:
import tensorflow_data_validation as tfdv

# Acceder y modificar la característica 'Hillshade_9am'
hillshade_9am = tfdv.get_feature(schema, 'Hillshade_9am')
hillshade_9am.int_domain.min = 0
hillshade_9am.int_domain.max = 255

# Acceder y modificar la característica 'Hillshade_Noon'
hillshade_noon = tfdv.get_feature(schema, 'Hillshade_Noon')
hillshade_noon.int_domain.min = 0
hillshade_noon.int_domain.max = 255

# Acceder y modificar la característica 'Slope'
slope_feature = tfdv.get_feature(schema, 'Slope')
slope_feature.int_domain.min = 0
slope_feature.int_domain.max = 90



In [25]:
# Acceder y modificar la característica 'Cover_Type'
cover_type_feature = tfdv.get_feature(schema, 'Cover_Type')
cover_type_feature.int_domain.min = 0
cover_type_feature.int_domain.max = 6

# Intentar marcar la característica como categórica.
# En algunas versiones de TFDV, 'is_categorical' no está disponible en int_domain.
try:
    cover_type_feature.int_domain.is_categorical = True
except AttributeError:
    print("La propiedad 'is_categorical' no está soportada en la versión actual de TFDV. " +
          "Se recomienda gestionar la variable categórica en el preprocesamiento o en el modelado.")

# Visualizar el esquema actualizado
tfdv.display_schema(schema)




,Type,Presence,Valency,Domain
Feature name,,,,
'Elevation',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Vertical_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Cover_Type',INT,required,,min: 0; max: 6


## 9. Detección de Anomalías y Análisis de Distribuciones

Se simula un conjunto de datos de servicio (escenario de inferencia) eliminando la etiqueta "Cover_Type" y se generan estadísticas para comparar con el esquema curado. Esto permite detectar discrepancias, como la ausencia de una columna esperada.

In [26]:
# --- 2.4. Validar datos de servicio (inferencia) ---
# Simular un conjunto de datos de servicio eliminando la etiqueta 'Cover_Type'
serving_df = test_df.drop(columns=['Cover_Type'])
stats_serving = tfdv.generate_statistics_from_dataframe(serving_df)

# Al validar, se detectará que falta la columna 'Cover_Type'
anomalies_serving = tfdv.validate_statistics(statistics=stats_serving, schema=schema)
tfdv.display_anomalies(anomalies_serving)

# Nota: Para ambientes de entrenamiento e inferencia se podría configurar el esquema de manera
# que la presencia de 'Cover_Type' sea requerida solo en entrenamiento y opcional en inferencia.

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


## Conclusión

En este notebook se ha:

- Ingresado y explorado el dataset de cubierta forestal.

- Seleccionado un subconjunto de características numéricas y la etiqueta.

- Dividido el dataset en conjuntos de entrenamiento y prueba.

- Generado estadísticas descriptivas con TFDV para analizar la distribución de los datos.

- Inferido un esquema y curado dicho esquema estableciendo dominios esperados para cada variable.

- Realizado una validación en un escenario de inferencia para detectar anomalías (por ejemplo, la ausencia de la columna "Cover_Type").

Este proceso permite asegurar la calidad de los datos y detectar posibles problemas antes de avanzar a etapas posteriores, como la transformación (con TensorFlow Transform) o el entrenamiento del modelo.
